# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [111]:
import os
import sys

sys.path.append('..')
from utils import *
# Set your preferred name
USER = "Yuxuan"
# Set the data and output directories
DATA_ROOT = '/home/yxcheng'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Head-and-Neck-Cancer'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [78]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [79]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Glioblastoma_(GBM)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Bladder_Cancer_(BLCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Cervical_Ca

If no match is found, jump directly to GEO in Part 2.2

In [80]:
trait_subdir = "TCGA_Head_and_Neck_Cancer_(HNSC)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.HNSC.sampleMap_HNSC_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.HNSC.sampleMap_HiSeqV2_PANCAN.gz')

In [81]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [82]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [83]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PANCAN_CNA_PANCAN_K8',
 '_PANCAN_Cluster_Cluster_PANCAN',
 '_PANCAN_DNAMethyl_HNSC',
 '_PANCAN_DNAMethyl_PANCAN',
 '_PANCAN_RPPA_PANCAN_K8',
 '_PANCAN_UNC_RNAseq_PANCAN_K16',
 '_PANCAN_miRNA_PANCAN',
 '_PANCAN_mirna_HNSC',
 '_PANCAN_mutation_PANCAN']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [84]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_HNSC', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_HNSC', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure',

In [85]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [86]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [66.0, 69.0, 49.0, 39.0, 45.0],
 'days_to_birth': [-24222.0, -25282.0, -17951.0, -14405.0, -16536.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, 462.0, 283.0, 415.0, 1134.0],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['102', nan, nan, nan, nan],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  396.0,
  236.0,
  286.0,
  nan],
 'stopped_smoking_year': [nan, nan, nan, nan, 2005.0],
 'year_of_initial_pathologic_diagnosis': [2013, 2003, 2004, 2003, 2003],
 'year_of_tobacco_smoking_onset': [nan, 1951.0, 1974.0, 1983.0, 1983.0]}

In [87]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [88]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}

In [89]:
gender_col = 'gender'

In [90]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [91]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

15 input query terms found dup hits:	[('GTF2IP1', 2), ('RBMY1A3P', 3), ('HERC2P2', 3), ('WASH3P', 3), ('NUDT9P1', 2), ('SNORD104', 2), ('
154 input query terms found no hit:	['C16orf13', 'C16orf11', 'LOC100272146', 'LOC339240', 'NACAP1', 'LOC441204', 'KLRA1', 'FAM183A', 'FA
9 input query terms found dup hits:	[('SUGT1P1', 2), ('SNORD127', 2), ('SNORA62', 4), ('IFITM4P', 8), ('HLA-DRB6', 2), ('FUNDC2P2', 2), 
190 input query terms found no hit:	['NARFL', 'NFKBIL2', 'LOC150197', 'TMEM84', 'LOC162632', 'PPPDE1', 'PPPDE2', 'C1orf38', 'C1orf31', '
20 input query terms found dup hits:	[('PIP5K1P1', 2), ('SNORA40', 2), ('SNORD116-24', 2), ('SNORD116-25', 2), ('SNORD116-23', 2), ('SNOR
149 input query terms found no hit:	['FAM153C', 'C9orf167', 'CLK2P', 'CCDC76', 'CCDC75', 'CCDC72', 'HIST3H2BB', 'PRAC', 'LOC285780', 'LO
18 input query terms found dup hits:	[('SNORD58C', 2), ('UOX', 2), ('UBE2Q2P1', 3), ('PPP4R1L', 2), ('SNORD63', 3), ('SNORD116-22', 2), (
158 input query terms found no hit:	['

In [92]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Head-and-Neck-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-4P-AA8J-01,1,66.0,1,-0.471492,0.808874,-0.531035,0.233028,-0.649078,-0.88201,-0.055394,...,-0.920186,-0.086682,0.409822,-1.587117,1.296567,3.68191,0.045223,0.847073,0.594105,0.690367
TCGA-BA-4074-01,1,69.0,1,-2.785992,0.931274,0.293365,-1.502772,0.422722,1.30079,-2.390494,...,-2.090786,-0.086682,-0.748878,-1.587117,-0.189333,1.77561,-0.278177,-0.298827,-0.615395,-2.848533
TCGA-BA-4075-01,1,49.0,1,-2.609592,-4.503726,1.544765,-0.871272,0.396522,1.27679,-1.614194,...,-2.090786,-0.086682,-0.129678,-1.587117,-0.209833,2.71931,-1.013777,-0.939227,-0.156195,-2.631833
TCGA-BA-4076-01,1,39.0,1,-0.524192,-4.519426,-0.531035,-0.015372,1.143422,0.26449,-1.286294,...,-2.090786,-0.086682,-0.140378,-1.587117,-2.422033,0.08271,-0.033777,0.080373,-1.184895,-3.472933
TCGA-BA-4077-01,1,45.0,0,0.019008,-1.699326,1.237165,-0.380472,0.060722,0.88759,-0.604294,...,0.511414,0.480118,-0.748878,-1.587117,0.994967,1.75751,-0.387677,-1.612027,-0.050895,-1.220433


In [93]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 565 samples.


In [94]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Head-and-Neck-Cancer', the least common label is '0' with 44 occurrences. This represents 7.79% of the dataset.
The distribution of the feature 'Head-and-Neck-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 53.0
  50% (Median): 61.0
  75%: 68.0
Min: 19.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 151 occurrences. This represents 26.73% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [95]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [96]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Head-and-Neck-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [4]:
dataset = 'GEO'
trait_subdir = "Head-and-Neck-Cancer"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE201777', 'GSE244580', 'GSE172159', 'GSE175509']

Repeat the below steps for all the accession numbers

In [15]:
cohort = accession_num = "GSE175509"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/home/yxcheng/GEO/Head-and-Neck-Cancer/GSE175509/GSE175509_family.soft.gz',
 '/home/yxcheng/GEO/Head-and-Neck-Cancer/GSE175509/GSE175509_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [16]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Integration of p16/HPV DNA status with a 24-miRNA-defined molecular phenotype improves clinically relevant stratification of head and neck cancer patients"
!Series_summary	"Head and neck squamous cell carcinomas (HNSCC) driven by human papillomavirus (HPV) generally have a more favourable prognosis. We hypothesized that HPV-positive HNSCC may be identified based on a miRNA signature according to their specific molecular pathogenesis and are characterized by a unique transcriptome compared to HPV-negative HNSCC."
!Series_summary	"We characterized the miRNA-expression patterns of the tumors from 229 head and neck squamous cell carcinoma patients by Agilent miRNA microarrays in order to define a HPV-predicting miRNA signature."
!Series_overall_design	"n=229 HNSCC tumor samples for which HPV-status was known were analyzed."


In [17]:
clinical_data.head()

,!Sample_geo_accession,GSM5336332,GSM5336333,GSM5336334,GSM5336335,GSM5336336,GSM5336337,GSM5336338,GSM5336339,GSM5336340,...,GSM5336551,GSM5336552,GSM5336553,GSM5336554,GSM5336555,GSM5336556,GSM5336557,GSM5336558,GSM5336559,GSM5336560
0,!Sample_characteristics_ch1,hpv-status: negative,hpv-status: negative,hpv-status: negative,hpv-status: negative,hpv-status: negative,hpv-status: negative,hpv-status: negative,hpv-status: positive,hpv-status: negative,...,hpv-status: negative,hpv-status: positive,hpv-status: positive,hpv-status: positive,hpv-status: positive,hpv-status: positive,hpv-status: positive,hpv-status: positive,hpv-status: positive,hpv-status: negative


In [18]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['hpv-status: negative', 'hpv-status: positive']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [12]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Psoriasis\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Psoriasis\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an intege

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [20]:
is_gene_availabe = True
trait_row_id = 1
age_row_id = 3
gender_row_id = 2

trait_type = 'binary'

In [14]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [15]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'primary diagnosis: PSORIASIS':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'Sex: Female':
        return 0
    elif gender_string == 'Sex: Male':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23y'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(0, None, None)

In [16]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:171: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:171: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:171: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


,GSM3494884,GSM3494885,GSM3494886,GSM3494887,GSM3494888,GSM3494889,GSM3494890,GSM3494891,GSM3494892,GSM3494893,...,GSM3495040,GSM3495041,GSM3495042,GSM3495043,GSM3495044,GSM3495045,GSM3495046,GSM3495047,GSM3495048,GSM3495049
Psoriasis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Age,56,None,20,51,37,61,None,31,56,41,...,20,69,69,47,47,56,54,53,50,22
Gender,1,None,0,0,1,1,None,1,0,0,...,1,1,0,1,1,0,0,0,0,1


### Genetic data preprocessing and final filtering

In [17]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,ID,GSM3494884,GSM3494885,GSM3494886,GSM3494887,GSM3494888,GSM3494889,GSM3494890,GSM3494891,GSM3494892,...,GSM3495040,GSM3495041,GSM3495042,GSM3495043,GSM3495044,GSM3495045,GSM3495046,GSM3495047,GSM3495048,GSM3495049
0,1,6.948572,6.783931,6.780049,7.159348,7.311038,8.522366,7.208509,7.339519,7.292977,...,6.957283,7.024298,6.953613,6.610053,6.493569,7.024804,7.469483,6.559963,6.866259,7.015812
1,2,4.267766,3.692029,3.649207,4.331090,3.903376,4.191000,3.771017,4.293601,3.963668,...,4.271395,4.137957,4.130696,3.941061,4.238553,3.597590,3.454440,3.558840,4.008234,3.686148
2,3,4.334513,3.981417,3.922257,4.549893,4.140639,4.013236,3.996610,4.933703,3.735861,...,4.303602,4.277528,4.234886,4.187910,4.143072,3.761361,3.791233,3.793128,3.344874,4.103369
3,9,7.140005,7.411580,6.722240,7.117540,6.874786,7.020385,7.031787,6.980904,7.003364,...,7.121634,6.912987,7.270386,7.561971,8.002471,7.195000,7.414168,7.845036,7.263841,7.301888
4,10,4.486670,4.615900,3.966261,4.543677,4.150289,4.216883,4.736946,4.296770,4.462828,...,4.162474,4.563109,3.977201,4.644588,4.734826,4.412295,4.422868,4.763507,5.003758,4.061337


In [18]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['1',
 '2',
 '3',
 '9',
 '10',
 '12',
 '13',
 '14',
 '15',
 '16',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [19]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1', '2', '3', '9', '10', '12', '13', '14', '15', '16', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27']\n"

If not required, jump directly to the gene normalization step

In [20]:
requires_gene_mapping = True

In [21]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['1', '2', '3', '9', '10'], 'ENTREZ_GENE_ID': ['1', '2', '3', '9', '10'], 'SPOT_ID': [1.0, 2.0, 3.0, 9.0, 10.0]}


Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [22]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['1', '2', '3', '9', '10'], 'ENTREZ_GENE_ID': ['1', '2', '3', '9', '10'], 'SPOT_ID': [1.0, 2.0, 3.0, 9.0, 10.0]}
    


In [23]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'ENTREZ_GENE_ID'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [24]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [25]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [26]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 154 samples.


In [27]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Psoriasis', the least common label is '1.0' with 11 occurrences. This represents 7.14% of the dataset.
The distribution of the feature 'Psoriasis' in this dataset is fine.

Quartiles for 'Age':
  25%: 26.0
  50% (Median): 47.0
  75%: 61.0
Min: 8.0
Max: 97.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 70 occurrences. This represents 45.45% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [28]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [29]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [97]:
from utils import *

In [98]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'Xena'

In [99]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,Xena,True,True,False,True,True,565,


In [100]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [101]:
merged_data.head()

,Head-and-Neck-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
0,1,66.0,1,-0.471492,0.808874,-0.531035,0.233028,-0.649078,-0.88201,-0.055394,...,-0.920186,-0.086682,0.409822,-1.587117,1.296567,3.68191,0.045223,0.847073,0.594105,0.690367
1,1,69.0,1,-2.785992,0.931274,0.293365,-1.502772,0.422722,1.30079,-2.390494,...,-2.090786,-0.086682,-0.748878,-1.587117,-0.189333,1.77561,-0.278177,-0.298827,-0.615395,-2.848533
2,1,49.0,1,-2.609592,-4.503726,1.544765,-0.871272,0.396522,1.27679,-1.614194,...,-2.090786,-0.086682,-0.129678,-1.587117,-0.209833,2.71931,-1.013777,-0.939227,-0.156195,-2.631833
3,1,39.0,1,-0.524192,-4.519426,-0.531035,-0.015372,1.143422,0.26449,-1.286294,...,-2.090786,-0.086682,-0.140378,-1.587117,-2.422033,0.08271,-0.033777,0.080373,-1.184895,-3.472933
4,1,45.0,0,0.019008,-1.699326,1.237165,-0.380472,0.060722,0.88759,-0.604294,...,0.511414,0.480118,-0.748878,-1.587117,0.994967,1.75751,-0.387677,-1.612027,-0.050895,-1.220433


In [102]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [103]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [104]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [105]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [106]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 53.10% ± 6.45%


In [107]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 82.48% ± 1.62%


In [108]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 5. Discussion and report

In [109]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0074
Found 28 genes with non-zero coefficients associated with the trait 'Head-and-Neck-Cancer' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
 Variable  Coefficient  Absolute Coefficient
     PAX2     1.034658              1.034658
      TNR    -0.798484              0.798484
    FFAR1     0.662355              0.662355
     ODF1     0.536018              0.536018
     RBP2     0.497310              0.497310
  KCNQ1DN    -0.332506              0.332506
      DDC     0.302512              0.302512
  DEFB125     0.243494              0.243494
   ZNF479    -0.231494              0.231494
 CEACAM20    -0.201933              0.201933
    AGTR2     0.189927              0.189927
    OR4S2     0.181811              0.181811
     SSX5    -0.180650              0.180650
     TAL1     0.170170              0.170170
LOC389705    -0.163031              0.163031


/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [110]:

interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0074
Found 25 significant genes associated with the trait 'Head-and-Neck-Cancer' conditional on the factor 'Age', with corrected p-value < 0.05:
   Variable  Coefficient      p_value  corrected_p_value
     ADAM12     0.078211 4.221327e-07           0.004089
      CIDEC    -0.040465 7.074216e-07           0.004089
       NID2     0.094251 6.558578e-07           0.004089
   DEFB105A     0.027188 5.195757e-06           0.019626
    FAM74A3    -0.023948 7.682262e-06           0.019626
     ZNF114     0.042696 6.605306e-06           0.019626
       HTN3    -0.033348 7.923346e-06           0.019626
     COL4A1     0.070817 1.616322e-05           0.023355
       ESM1     0.040382 1.264265e-05           0.023355
     COL5A2     0.074851 1.261872e-05           0.023355
    PLA2G2E    -0.025294 1.582130e-05           0.023355
      MUC21     0.046243 1.421064e-05           0.023355
  C14orf180    -0.038985 1.874611e-0

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
